In [1]:
import functools

import numpy as np
import tensorflow as tf

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [4]:
# numpyの値を読みやすくする
np.set_printoptions(precision=3, suppress=True)

In [5]:
LABELS = [0, 1]
LABEL_COLUMN = "survived"

In [8]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name = LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    return dataset

def show_batch(dataset):
    for batch, label in dataset.take(1): #labelどこで使ってる？？
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

In [9]:
raw_train_data = get_dataset(train_file_path) #形がわからん
raw_test_data = get_dataset(test_file_path)

In [10]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'female' b'male']
age                 : [28. 28. 31. 27. 36.]
n_siblings_spouses  : [0 0 0 1 1]
parch               : [0 0 0 0 2]
fare                : [ 26.55   7.75  10.5   21.   120.  ]
class               : [b'First' b'Third' b'Second' b'Second' b'First']
deck                : [b'C' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'n']


In [13]:
# データセットの読み込み方法２
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)

sex                 : [b'female' b'male' b'male' b'female' b'male']
age                 : [58. 36. 28.  9. 30.]
n_siblings_spouses  : [0 0 0 2 0]
parch               : [0 0 0 2 0]
fare                : [146.521  26.388  13.     34.375   7.225]
class               : [b'First' b'First' b'Second' b'Third' b'Third']
deck                : [b'B' b'E' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'y' b'n' b'y']


In [14]:
# 特定ののカラムだけ読み込む方法
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS) #select_columnsで特定のカラムだけ抽出できる
show_batch(temp_dataset)

age                 : [19. 71. 21. 23. 30.]
n_siblings_spouses  : [0 0 0 2 0]
class               : [b'Third' b'First' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'A' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'y' b'y' b'n' b'y']


In [15]:
# survivedがどっか行ってる、なんで？？
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)
show_batch(temp_dataset)

age                 : [11. 44. 24. 36. 35.]
n_siblings_spouses  : [0. 0. 0. 1. 0.]
parch               : [0. 1. 0. 2. 0.]
fare                : [ 18.788  57.979  13.    120.     10.5  ]


In [17]:
# 数値列をベクトル化
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [18]:
packed_dataset = temp_dataset.map(pack)
for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[ 49.      1.      0.     89.104]
 [ 24.      0.      3.     19.258]
 [ 50.      0.      1.    247.521]
 [ 28.      0.      0.      7.879]
 [ 49.      1.      1.    110.883]]

[1 1 1 1 0]


In [19]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'female']
age                 : [28. 28. 24. 23. 19.]
n_siblings_spouses  : [0 0 0 0 1]
parch               : [0 0 0 0 0]
fare                : [ 7.25   8.113  8.05  10.5   26.   ]
class               : [b'Third' b'Third' b'Third' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'n']


In [21]:
example_batch, labels_batch = next(iter(temp_dataset))

In [23]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features

        return features, labels

In [24]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [25]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'female']
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'n' b'y' b'y']
numeric             : [[20.     0.     0.     7.229]
 [24.     0.     0.     7.896]
 [18.     0.     1.     9.35 ]
 [55.5    0.     0.     8.05 ]
 [15.     0.     0.     7.225]]


In [26]:
example_batch, labels_batch = next(iter(packed_train_data))

## データの正規化

In [30]:
import pandas as pd
# NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare'] すでに定義されてある
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [32]:
MEAN = np.array(desc.T["mean"]) #DataFrame.T は転置
STD = np.array(desc.T["std"])

In [33]:
# 何を表している？？　A.mean nomalization 正規化
def normalize_numeric_data(data, mean, std):
    return (data-mean)/std

In [34]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD) #normalize_numeric_dataの第一引数以外を固定
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x0000013D447BCAF8>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [35]:
example_batch["numeric"]

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 28.   ,   3.   ,   1.   ,  25.467],
       [ 30.   ,   0.   ,   0.   ,   7.225],
       [ 26.   ,   2.   ,   0.   ,   8.663],
       [ 17.   ,   1.   ,   0.   , 108.9  ],
       [ 28.   ,   0.   ,   0.   ,   0.   ]], dtype=float32)>

In [36]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_column)
numeric_layer(example_batch).numpy()

array([[-0.13 ,  2.132,  0.782, -0.163],
       [ 0.029, -0.474, -0.479, -0.497],
       [-0.29 ,  1.264, -0.479, -0.471],
       [-1.01 ,  0.395, -0.479,  1.365],
       [-0.13 , -0.474, -0.479, -0.63 ]], dtype=float32)

## カテゴリデータ

In [37]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [39]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [40]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [41]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [43]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13   2.132
  0.782 -0.163  1.     0.   ]


## モデルの構築

In [44]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

## 訓練・予測

In [45]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [46]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 2s 15ms/step - loss: 0.5138 - accuracy: 0.7337
Epoch 2/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4329 - accuracy: 0.8198
Epoch 3/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4078 - accuracy: 0.8278
Epoch 4/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3901 - accuracy: 0.8278
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3795 - accuracy: 0.8373
Epoch 6/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3676 - accuracy: 0.8469
Epoch 7/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3622 - accuracy: 0.8341
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3570 - accuracy: 0.8469
Epoch 9/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3533 - accuracy: 0.8501
Epoch 10/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3430 - accuracy: 0.851

In [47]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     53/Unknown - 1s 14ms/step - loss: 0.4522 - accuracy: 0.8485

Test Loss 0.4522127339721851, Test Accuracy 0.8484848737716675


In [49]:
predictions = model.predict(test_data)

# 結果のいくつかを表示
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: -125.28%  | Actual outcome:  SURVIVED
Predicted survival: -155.00%  | Actual outcome:  SURVIVED
Predicted survival: -51.26%  | Actual outcome:  DIED
Predicted survival: -180.58%  | Actual outcome:  DIED
Predicted survival: 42.58%  | Actual outcome:  DIED
